# Workshop 2: Connectivity principles and structure function analysis

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
<font size=4> The function of the nervous system arises out of a combination of the properties of individual neurons and the properties of how they are connected into a larger network. The central goal of <b>connectomics</b> is to produce complete maps of the connectivity of the nervous system with synaptic resolution and analyze them to better understand the organization, development, and function of the nervous system. </font>

<font size=4><b>Electron Microscopy (EM) data enables morphological reconstruction of neurons and resolution of their synaptic connectivity </b>. The V1DD dataset is one of the largest volume EM datasets currently available, and spans all layers of mouse visual cortex. We will be using this dataset to query the connectivity between neurons in the visual cortex. </font>

<b> The V1DD dataset is newly public. </b> Previous years of this course have used the [MICrONS Dataset](https://www.microns-explorer.org/), which is also a cubic millimeter volume of mouse visual cortex. Both datasets are available for the use of this course, but we will cover V1DD in Week 1.

<h3> Workshop aims</h3>
<ul>
    <li> Understand what are the major classes of cell types in cortex</li>
    <li> Understand the basics of how synaptic connectivity is measured in EM connectomics</li>
    <li> Look at the relationship between the structure of networks and cell type connectivity</li>
    <li> Look at the relationship between functional tuning properties of neurons and how they are connected in the network </li>
</ul>


In this workshop, we explore different two principles that govern how neurons are connected with each other. We will look how

<ol> 
    <li>spatial distance between neurons</li>
    <li>correlation of acitivity traces during specific stimuli </li>
</ol>

are related to neuron-to-neuron connectivity. We will then use the insigts about the relationship between neuron-neuron distance and connectivity to build a new NULL model for the structure-function analysis.

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

This cell sets up a variable called `data_root` that generalizes the data directory across systems, whether you are using CodeOcean or the prepared data drives

</div>

In [ ]:
import sys
from os.path import join as pjoin
import platform

# Add the directory for the data and utilities
mat_version = 1196

platstring = platform.platform()
system = platform.system()
if system == "Darwin":
    # macOS
    data_root = "/Volumes/Brain2025/"
elif system == "Windows":
    # Windows (replace with the drive letter of USB drive)
    data_root = "E:/"
elif "amzn" in platstring:
    # then on CodeOcean
    data_root = "/data/"
else:
    # then your own linux platform
    # EDIT location where you mounted hard drive
    data_root = "/media/$USERNAME/Brain2025/"

# Set the directory to load prepared data and utility code
data_dir = pjoin(data_root, f"v1dd_{mat_version}")
utils_dir = pjoin("..", "utils")

# Add utilities to path
sys.path.append(utils_dir)

from utils import filter_synapse_table

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<h2> Import packages </h2>

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import itertools
from typing import Union, Optional
from scipy import stats, spatial
import itertools
import tqdm
from os.path import join as pjoin
import collections

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

## Preparation

First, we are loading data files that were introduced in the first workshop such as the synapse and cell tables, and information about proofreading. This workshop does not require the CAVEclient.

In [ ]:
# Loads cells with axon and dendrite proofreading
dendrite_proof_root_ids = np.load(pjoin(data_dir, f'proofread_dendrite_list_{mat_version}.npy'))
axon_proof_root_ids = np.load(pjoin(data_dir, f'proofread_axon_list_{mat_version}.npy'))

print(f'There are {len(dendrite_proof_root_ids)} cells with acceptable dendrites, and {len(axon_proof_root_ids)} cells with axon proofreading')
print(f'The number of cells with usable axons and dendrites is: {len(np.isin(axon_proof_root_ids, dendrite_proof_root_ids))}')

In [ ]:
syn_df = pd.read_feather(f"{data_dir}/syn_df_all_to_proofread_to_all_{mat_version}.feather")
cell_df = pd.read_feather(f"{data_dir}/soma_and_cell_type_{mat_version}.feather")

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

The cell dataframe contains coordinates for all cells in nanometers. For convenience, we convert those to µm.

In [ ]:
cell_df[[col for col in cell_df.columns if "position" in col]] /= 1000.

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

Let's remind us of the cell types available to us

In [ ]:
cell_df["cell_type"].value_counts()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

## #1: Connection probability vs distance

In this section we will relate connectivity to the distance between two neurons, that is the distance between their soma. We will restrict our analysis to L3-IT pyramidal neurons.

In [ ]:
sub_cell_df = cell_df[cell_df["cell_type"] == "L3-IT"]

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

The dataset was not imaged in a perfectly upright position. We corrected for that and created the `*trform*` columns for each neuron position. We will use these for our analysis. However, to find neurons in neuroglancer, one has to still use the not-transformed positions.

In [ ]:
sub_cell_df.head(5)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

ax.scatter(data=sub_cell_df, x="pt_position_trform_x", y="pt_position_trform_z", c="k", s=1)

ax.tick_params(axis='both', which='major', labelsize=15, width=2, length=8)  # Set larger tick labels
ax.tick_params(axis='both', which='minor', labelsize=14, width=1, length=4)  # Set larger tick labels

ax.spines['top'].set_visible(False)  # Hide the top spine
ax.spines['right'].set_visible(False)  # Hide the right spine
ax.spines['bottom'].set_linewidth(2)  # Hide the top spine
ax.spines['left'].set_linewidth(2)  # Hide the right spine

ax.set_xlabel("Soma location x [µm]", fontsize=15)
ax.set_ylabel("Soma location z [µm]", fontsize=15)

plt.show()

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

<p><b>Task:</b> Are the proofread neurons evenly distributed throughout the dataset? Filter the "sub_cell_df" down to just those cells with proofread axons and plot them on top of the locations of all L3-IT cells.

</div>

In [ ]:
# Hint: keep ids that are also in by axon_proof_root_ids

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

To test whether distance and connectedness are correlated, we calculate the _lateral_ distance between pairs of neurons. The _lateral_ distance ignores cortical depth. For a pair of neurons the lateral distance can be calculated as the norm of the difference of their coordinates (in µm):

In [ ]:
pos_cols = ["pt_position_trform_x", "pt_position_trform_z"]

difference = sub_cell_df.iloc[0][pos_cols] - sub_cell_df.iloc[1][pos_cols]
np.linalg.norm(difference)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

This function computes the lateral distances across all neurons efficiently.

In [ ]:
def calculate_lateral_distances(pre_cell_df, post_cell_df=None):
    """ Calculates the lateral distances in µm between all neurons."""
    if post_cell_df is None:
        post_cell_df = pre_cell_df
    
    pre_lateral_locations = np.array(pre_cell_df[["pt_position_trform_x", "pt_position_trform_z"]])
    post_lateral_locations = np.array(post_cell_df[["pt_position_trform_x", "pt_position_trform_z"]])
    lateral_distances = spatial.distance.cdist(pre_lateral_locations, post_lateral_locations)
    
    id_pairs = list(itertools.product(pre_cell_df["pt_root_id"], post_cell_df["pt_root_id"]))
    
    lateral_distance_df = pd.DataFrame(id_pairs, columns=['pre_pt_root_id', 'post_pt_root_id'])
    lateral_distance_df['distance'] = lateral_distances.flatten()

    lateral_distance_df = lateral_distance_df[lateral_distance_df['pre_pt_root_id'] != lateral_distance_df['post_pt_root_id']]
    return lateral_distance_df

In [ ]:
# This will take a minute to complete
lateral_distance_df = calculate_lateral_distances(sub_cell_df)
lateral_distance_df.head(5)

In [ ]:
bins = np.linspace(0, 1400, 141)

fig, ax = plt.subplots(figsize=(8, 5))

sns.histplot(lateral_distance_df, x="distance", fill=True, bins=bins, alpha=.5, color="k", lw=3, element="step")

ax.tick_params(axis='both', which='major', labelsize=15, width=2, length=8)  
ax.tick_params(axis='both', which='minor', labelsize=14, width=1, length=4)  

ax.spines['top'].set_visible(False) 
ax.spines['right'].set_visible(False) 
ax.spines['bottom'].set_linewidth(2)  
ax.spines['left'].set_linewidth(2) 

ax.set_xlabel("Lateral distance [µm]", fontsize=15)
ax.set_ylabel("Neuron pair count", fontsize=15)

plt.show()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

This distribution shows the distance between _all pairs_ of L3-IT neurons in the dataset. However, these are not all the pairs for which we can probe connectivity, because not all of them have been proofread. Next, we plot the distances for just the pairs for which we have proofreading information available.

In [ ]:
# Gathering proofread L3-IT IDs
pre_root_ids = sub_cell_df["pt_root_id"][np.isin(sub_cell_df["pt_root_id"], axon_proof_root_ids)]
post_root_ids = sub_cell_df["pt_root_id"][np.isin(sub_cell_df["pt_root_id"], dendrite_proof_root_ids)]

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

Next, we filter the distance dataframe and comparing it to the unfiltered version.

In [ ]:
lateral_distance_proofread_df = lateral_distance_df.copy()
lateral_distance_proofread_df = lateral_distance_proofread_df[np.isin(lateral_distance_proofread_df["pre_pt_root_id"], pre_root_ids)]
lateral_distance_proofread_df = lateral_distance_proofread_df[np.isin(lateral_distance_proofread_df["post_pt_root_id"], post_root_ids)]

In [ ]:
bins = np.linspace(0, 1400, 141)

fig, ax = plt.subplots(figsize=(8, 5))

sns.histplot(lateral_distance_df, x="distance", fill=True, bins=bins, alpha=.5, color="k", lw=3, element="step", label="all pairs")
sns.histplot(lateral_distance_proofread_df, x="distance", fill=True, bins=bins, alpha=.5, color="b", lw=3, element="step", label="proofread pairs")
ax.legend(frameon=False, fontsize=14)

ax.tick_params(axis='both', which='major', labelsize=15, width=2, length=8)  
ax.tick_params(axis='both', which='minor', labelsize=14, width=1, length=4)  

ax.spines['top'].set_visible(False) 
ax.spines['right'].set_visible(False) 
ax.spines['bottom'].set_linewidth(2)  
ax.spines['left'].set_linewidth(2) 

ax.set_xlabel("Lateral distance [µm]", fontsize=15)
ax.set_ylabel("Neuron pair count", fontsize=15)

plt.show()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

We change to fractions to adjust for the different counts in the populations:

In [ ]:
bins = np.linspace(0, 1400, 141)

fig, ax = plt.subplots(figsize=(8, 5))

sns.histplot(lateral_distance_df, stat="proportion", x="distance", fill=True, bins=bins, alpha=.5, color="k", lw=3, element="step", label="all pairs")
sns.histplot(lateral_distance_proofread_df, stat="proportion", x="distance", fill=True, bins=bins, alpha=.5, color="b", lw=3, element="step", label="proofread pairs")
ax.legend(frameon=False, fontsize=14)

ax.tick_params(axis='both', which='major', labelsize=15, width=2, length=8)  
ax.tick_params(axis='both', which='minor', labelsize=14, width=1, length=4)  

ax.spines['top'].set_visible(False) 
ax.spines['right'].set_visible(False) 
ax.spines['bottom'].set_linewidth(2)  
ax.spines['left'].set_linewidth(2) 

ax.set_xlabel("Lateral distance [µm]", fontsize=15)
ax.set_ylabel("Fraction of neuron pairs", fontsize=15)

plt.show()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

Next, we want to compare this distribution with the distances between the all _connected_ neuron pairs to see if they differ. For that, we filter the synapse table to only include the proofread L3-IT neurons.

In [ ]:
analysis_syn_df = filter_synapse_table(syn_df, pre_root_ids, post_root_ids).reset_index(drop=True)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

Some neurons are connected with multiple synapses. Here, we will add them together to a single connection.

</div>

In [ ]:
conn_df = analysis_syn_df.groupby(["pre_pt_root_id", "post_pt_root_id"])["size"].sum().reset_index()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

Finally, we need to combine distance and connectivity information. We have this information in two separate dataframes:

In [ ]:
conn_df.head(5)

In [ ]:
lateral_distance_df.head(5)

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

<p><b>Task:</b> For each connection, we would like to know what the distance is between the two neurons. Write the pd.merge function to combine the distance and synapse information of the two tables. Note that the connectivity table has far fewer entries than the distance table. By merging them, we can filter the distance table down to the neuron pairs for which we have a synaptic connection.

Hint: pd.merge can merge on multiple columns at the same. 

</div>

In [ ]:
# merge conn_df and lateral_distance_df


In [ ]:
conn_dist_df.shape, conn_df.shape, lateral_distance_df.shape

In [ ]:
bins = np.linspace(0, 1400, 141)

fig, ax = plt.subplots(figsize=(8, 5))

sns.histplot(lateral_distance_df, stat="proportion", x="distance", fill=True, bins=bins, alpha=.5, color="k", lw=3, element="step", label="all pairs")
sns.histplot(lateral_distance_proofread_df, stat="proportion", x="distance", fill=True, bins=bins, alpha=.5, color="b", lw=3, element="step", label="proofread pairs")
sns.histplot(conn_dist_df, x="distance", stat="proportion", bins=bins, element="step", lw=3, fill=True, alpha=.5, color="r", label="connected pairs")
ax.legend(frameon=False, fontsize=14)

ax.tick_params(axis='both', which='major', labelsize=15, width=2, length=8)  
ax.tick_params(axis='both', which='minor', labelsize=14, width=1, length=4)  

ax.spines['top'].set_visible(False) 
ax.spines['right'].set_visible(False) 
ax.spines['bottom'].set_linewidth(2)  
ax.spines['left'].set_linewidth(2) 

ax.set_xlabel("Lateral distance [µm]", fontsize=15)
ax.set_ylabel("Fraction of neuron pairs", fontsize=15)

plt.show()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

To get a better intution, we will look at some examples of connected neuron pairs across distance:

- 10µm: https://spelunker.cave-explorer.org/#!middleauth+https://global.daf-apis.com/nglstate/api/v1/5054147278667776
- 50µm: https://spelunker.cave-explorer.org/#!middleauth+https://global.daf-apis.com/nglstate/api/v1/5895810897149952
- 100µm: https://spelunker.cave-explorer.org/#!middleauth+https://global.daf-apis.com/nglstate/api/v1/5101346838544384
- 500µm: https://spelunker.cave-explorer.org/#!middleauth+https://global.daf-apis.com/nglstate/api/v1/6458760850571264

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

<p><b>Task:</b> Find the synapse between the 500µm-neuron pair.

</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

Another way to represent the data above is to calculate a "connection probability". That is, <b>how likely is it that two neurons are connected given their lateral distance from each other? </b> 

There are different ways to achieve this, here we will calculate the connection probability for every distance bin.

In [ ]:
bins = np.linspace(0, 500, 101) # 5µm bins

# Binned counts for proofread and connected pairs
proofread_pair_counts = np.histogram(lateral_distance_proofread_df["distance"], bins)[0]
connected_pair_counts = np.histogram(conn_dist_df["distance"], bins)[0]

# Fraction of counts between connected and proofread pairs
connection_probability = connected_pair_counts / proofread_pair_counts
connection_probability_df = pd.DataFrame({"bin_id": np.arange(len(bins)-1), "probability": connection_probability})

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

sns.lineplot(x=bins[:-1], y=connection_probability, lw=3, color="k")

ax.tick_params(axis='both', which='major', labelsize=15, width=2, length=8)  # Set larger tick labels
ax.tick_params(axis='both', which='minor', labelsize=14, width=1, length=4)  # Set larger tick labels

ax.spines['top'].set_visible(False)  # Hide the top spine
ax.spines['right'].set_visible(False)  # Hide the right spine
ax.spines['bottom'].set_linewidth(2)  # Hide the top spine
ax.spines['left'].set_linewidth(2)  # Hide the right spine

ax.set_xlabel("Lateral distance [µm]", fontsize=15)
ax.set_ylabel("Connection probability", fontsize=15)

sns.despine(offset=15, trim=False)
plt.tight_layout()
plt.show()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

## #2: Structure vs function: Connectivity vs activity correlation

The V1dd dataset is a structure function dataset. We have functional measurements for many cells in the datasets. These are very rich and can be related to behavioral data. 

You have seen data access instructions for nwb files previously in this course. For V1DD, see data instructions in the databook:

<font size=4> <a href=https://allenswdb.github.io/physiology/ophys/V1DD/V1DD-overview.html> V1 Deep Dive Overview </a> </font>

</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<b>Activity correlations</b> For the purpose of this workshop, we precalculated correlations between cells. To be more precise, we calculated the total correlation of the activity traces of neurons imaged during the same session during the same stimulus condition.

If you want to use the V1dd dataset for a project, this kind of analysis could be extended to include behavioral data and create more sophisticated functional comparisons between cells which can then be related to the connectivity data.

<b>Coregistration</b>: To analyze the functional and connectivity data, neurons in both datasets need to be _coregistered_, that is an ROI in the activity stack needs to be linked with a cell body in the EM stack. The data below only contains those neurons which were coregistered.

In [ ]:
corr_coreg_df = pd.read_feather(f"{data_dir}/cell_cell_correlations_by_stimulus_coregistered.feather")

In [ ]:
corr_coreg_df.head(5)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

Correlations of activity traces for different stimulus conditions are available to us.

In [ ]:
stimulus_conditions = corr_coreg_df.columns[3:]
stimulus_conditions

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

Can we treat the different stimulus conditions as separate experiments or would we expect all results to be similar no matter what stimulus condition we base or analysis on? Next, we analyze how correlated the co-activities of cells are between the different stimulus conditions. For that, we calculate the correlation of the pairwise correlations. 

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

<p><b>Discussion:</b> What outcome do you expect and what do different results mean for the use of the neuron-neuron correlations for further analysis?

</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

We use pandas [corr](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html) function to correlated all pairs of stimulus condition columns:

In [ ]:
stim_stim_corr_df = corr_coreg_df[stimulus_conditions].corr()
stim_stim_corr_df

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4), dpi=200)

heatmap = sns.heatmap(stim_stim_corr_df, ax=ax, vmax=.5, vmin=0)
colorbar = heatmap.collections[0].colorbar
colorbar.set_label("Correlation of neuron-neuron corrs.", fontsize=15)

ax.tick_params(axis='both', which='major', labelsize=16, width=2, length=8)  # Set larger tick labels
ax.tick_params(axis='both', which='minor', labelsize=14, width=1, length=4)  # Set larger tick labels

ax.spines['top'].set_visible(False)  # Hide the top spine
ax.spines['right'].set_visible(False)  # Hide the right spine
ax.spines['bottom'].set_linewidth(2)  # Hide the top spine
ax.spines['left'].set_linewidth(2)  # Hide the right spine

ax.set_xticklabels([])
ax.set_xlabel("")
ax.set_ylabel("")
sns.despine(offset=15, trim=False)
plt.tight_layout()
plt.show()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

Another way to compare the different stimulus conditions is to plot the distributions of the neuron-neuron correlations.

In [ ]:
stim = stimulus_conditions[2]

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4), dpi=200)

sns.histplot(data=corr_coreg_df, x=stim, label=stim, bins=np.linspace(-1, 1, 201), 
             element="step", lw=3, ax=ax, stat="probability", alpha=.5)

ax.legend(frameon=False)
ax.tick_params(axis='both', which='major', labelsize=16, width=2, length=8)  # Set larger tick labels
ax.tick_params(axis='both', which='minor', labelsize=14, width=1, length=4)  # Set larger tick labels

ax.spines['top'].set_visible(False)  # Hide the top spine
ax.spines['right'].set_visible(False)  # Hide the right spine
ax.spines['bottom'].set_linewidth(2)  # Hide the top spine
ax.spines['left'].set_linewidth(2)  # Hide the right spine
ax.set_xlim(-.1, .3)

ax.set_xlabel("Total trace correlation during stimulus", fontsize=16)
ax.set_ylabel("Fraction of neuron-pairs", fontsize=16)
sns.despine(offset=15, trim=False)
plt.tight_layout()
plt.show()

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

<p><b>Task:</b> Add the distributions for the other stimulus conditions into the same plot.

</div>

In [ ]:
# Plot for stim in stimulus_conditions

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

With the activity correlation between neurons in hand, we want to answer one question: <b>Are neurons that are more correlated in their activity, more likely to be connected with each other?</b>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

As before, we will limit this analysis to L3-IT neurons. We collect the connectivity information between these neurons as before.

In [ ]:
sub_cell_df = cell_df[cell_df["cell_type"] == "L3-IT"]

In [ ]:
pre_root_ids = sub_cell_df["pt_root_id"][np.isin(sub_cell_df["pt_root_id"], axon_proof_root_ids)]
post_root_ids = sub_cell_df["pt_root_id"][np.isin(sub_cell_df["pt_root_id"], dendrite_proof_root_ids)]

In [ ]:
analysis_syn_df = filter_synapse_table(syn_df, axon_proof_root_ids, dendrite_proof_root_ids).reset_index(drop=True)
conn_df = analysis_syn_df.groupby(["pre_pt_root_id", "post_pt_root_id"])["size"].sum().reset_index()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

We will limit our analysis to just those neurons for which we have proofreading data available

In [ ]:
corr_coreg_proofread_df = corr_coreg_df.copy()
corr_coreg_proofread_df = corr_coreg_proofread_df[np.isin(corr_coreg_proofread_df["pre_pt_root_id"], pre_root_ids)]
corr_coreg_proofread_df = corr_coreg_proofread_df[np.isin(corr_coreg_proofread_df["post_pt_root_id"], post_root_ids)]

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

Let's compare the connectivity and the activity correlation data.

In [ ]:
conn_corr_df = pd.merge(conn_df, corr_coreg_proofread_df, on=["pre_pt_root_id", "post_pt_root_id"])

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

These are the numbers of neuron pairs we have available for our comparison:

In [ ]:
corr_coreg_proofread_df.shape, conn_corr_df.shape

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

For a given stimulus, we compare the distributions of activity correlations between the connected pairs with the activity correlations of all pairs.

In [ ]:
# Select a stimulus condition to plot
stim = stimulus_conditions[2]
stim

In [ ]:
bins=np.linspace(-.1, .4, 26)

fig, ax = plt.subplots(figsize=(8, 5))

ax.set_title(stim, fontsize=18)

sns.histplot(corr_coreg_proofread_df, x=stim, stat="probability", bins=bins, element="step", label="proofread pairs", lw=3, fill=True, alpha=.5, color="k")
sns.histplot(conn_corr_df, x=stim, stat="probability", bins=bins, element="step", label="connected pairs", lw=3, fill=True, alpha=.5, color="r")

ax.legend(frameon=False, fontsize=14)

ax.tick_params(axis='both', which='major', labelsize=15, width=2, length=8)  # Set larger tick labels
ax.tick_params(axis='both', which='minor', labelsize=14, width=1, length=4)  # Set larger tick labels

ax.spines['top'].set_visible(False)  # Hide the top spine
ax.spines['right'].set_visible(False)  # Hide the right spine
ax.spines['bottom'].set_linewidth(2)  # Hide the top spine
ax.spines['left'].set_linewidth(2)  # Hide the right spine

ax.set_xlabel("Total trace correlation during stimulus", fontsize=16)
ax.set_ylabel("Fraction of neuron pairs", fontsize=15)
sns.despine(offset=15, trim=False)
plt.tight_layout()

plt.show()

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

<p><b>Task:</b> Repeat the plot above for different stimulus conditions. Do they all show similar results?

</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

Just like for the distance analysis above (#1), we can express this result as a "connection probability" for a given correlation. That is, how likely is it that two neurons are connected given their activity correlation? 

Here, we will calculate the connection probability bins of activity correlations. We use percentile bins to avoid having to pick suitable bin locations.

In [ ]:
bins = np.percentile(corr_coreg_proofread_df[stim], np.linspace(0, 100, 6)) # Makes 5 bins
bin_centers = np.convolve(bins, [.5, .5], mode="valid") # Calulates the center of each bin

# Counts for proofread and connected pairs
proofread_pair_counts = np.histogram(corr_coreg_proofread_df[stim], bins)[0]
connected_pair_counts = np.histogram(conn_corr_df[stim], bins)[0]

# Fraction of counts between connected and proofread pairs
connection_probability = connected_pair_counts / (proofread_pair_counts + 1e-6)

In [ ]:
fig, ax = plt.subplots(figsize=(7, 5))

sns.lineplot(x=bin_centers, y=connection_probability, lw=3, ls="--", legend=False, ax=ax, marker="o", markersize=15)

ax.legend(frameon=False, fontsize=14)
ax.get_legend().set_bbox_to_anchor([1.1, 1])
ax.tick_params(axis='both', which='major', labelsize=15, width=2, length=8)  # Set larger tick labels
ax.tick_params(axis='both', which='minor', labelsize=14, width=1, length=4)  # Set larger tick labels

ax.spines['top'].set_visible(False)  # Hide the top spine
ax.spines['right'].set_visible(False)  # Hide the right spine
ax.spines['bottom'].set_linewidth(2)  # Hide the top spine
ax.spines['left'].set_linewidth(2)  # Hide the right spine

ax.set_xlabel("Total trace correlation during stimulus", fontsize=16)
ax.set_ylabel("Connection probability", fontsize=15)

sns.despine(offset=15, trim=False)
plt.tight_layout()
plt.show()

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

<p><b>Task:</b> Add the distributions for the other stimulus conditions into the same plot.

</div>

In [ ]:
# Plot for stim in stimulus_conditions

# Solution
fig, ax = plt.subplots(figsize=(11, 5))

for stim in stimulus_conditions:
    bins = np.percentile(corr_coreg_proofread_df[stim], np.linspace(0, 100, 6))#[[0, 1, -2, -1]]

    bin_centers = np.convolve(bins, [.5, .5], mode="valid")#[[0, -1]]
    proofread_pair_counts = np.histogram(corr_coreg_proofread_df[stim], bins)[0]#[[0, -1]]
    connected_pair_counts = np.histogram(conn_corr_df[stim], bins)[0]#[[0, -1]]
    
    connection_probability = connected_pair_counts / (proofread_pair_counts + 1e-6)
    
    sns.lineplot(x=bin_centers, y=connection_probability, lw=3, ls="--", label=stim, legend=False, ax=ax, marker="o", markersize=15)

ax.legend(frameon=False, fontsize=14)
ax.get_legend().set_bbox_to_anchor([1.1, 1])
ax.tick_params(axis='both', which='major', labelsize=15, width=2, length=8)  # Set larger tick labels
ax.tick_params(axis='both', which='minor', labelsize=14, width=1, length=4)  # Set larger tick labels

ax.spines['top'].set_visible(False)  # Hide the top spine
ax.spines['right'].set_visible(False)  # Hide the right spine
ax.spines['bottom'].set_linewidth(2)  # Hide the top spine
ax.spines['left'].set_linewidth(2)  # Hide the right spine

ax.set_xlabel("Total trace correlation during stimulus", fontsize=16)
ax.set_ylabel("Connection probability", fontsize=15)

sns.despine(offset=15, trim=False)
plt.tight_layout()
plt.show()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

### Summary

We showed that for L3-IT cells

1. neurons that are closer to each other, are more likely to be connected.
2. neurons with more correlated activity traces, are more likely to be connected.

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

## A stronger NULL model: Can distance $\rightarrow$ connectivity account for structure-function correlation

So far we implicity used a _NULL model_ where all neuron pairs have the same likelihood to be connected. But we just showed that spatial separation alone increases the probability that two neurons are connected. Can this account for the correlation between activity correlation and connectivity?

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

<p><b>Discuss:</b> How could we implement such a NULL model?

</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<b>Collecting data products:</b> We will create a single dataframe that contains all information about the proofread, coregistered pairs: Their distance, their activity correlations, whether they are connected. Let's port the results from our previous analyses

In [ ]:
# Select one subtype of cell
sub_cell_df = cell_df[cell_df["cell_type"] == "L3-IT"]

pre_root_ids = sub_cell_df["pt_root_id"][np.isin(sub_cell_df["pt_root_id"], axon_proof_root_ids)]
post_root_ids = sub_cell_df["pt_root_id"][np.isin(sub_cell_df["pt_root_id"], dendrite_proof_root_ids)]

In [ ]:
# Connectivity analaysis

analysis_syn_df = filter_synapse_table(syn_df, axon_proof_root_ids, dendrite_proof_root_ids).reset_index(drop=True)
conn_df = analysis_syn_df.groupby(["pre_pt_root_id", "post_pt_root_id"])["size"].sum().reset_index()

In [ ]:
# Distance analysis

lateral_distance_df = calculate_lateral_distances(sub_cell_df)

lateral_distance_proofread_df = lateral_distance_df.copy()
lateral_distance_proofread_df = lateral_distance_proofread_df[np.isin(lateral_distance_proofread_df["pre_pt_root_id"], pre_root_ids)]
lateral_distance_proofread_df = lateral_distance_proofread_df[np.isin(lateral_distance_proofread_df["post_pt_root_id"], post_root_ids)]

In [ ]:
# Activity correlation

corr_coreg_proofread_df = corr_coreg_df.copy()
corr_coreg_proofread_df = corr_coreg_proofread_df[np.isin(corr_coreg_proofread_df["pre_pt_root_id"], pre_root_ids)]
corr_coreg_proofread_df = corr_coreg_proofread_df[np.isin(corr_coreg_proofread_df["post_pt_root_id"], post_root_ids)]

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

**Combine data products**: Let's combine our main three dataframes into one. We are merging the dataframes together, in a way that retains all entries in "corr_coreg_proofread_df". 

In [ ]:
# set 'corr_coreg_proofread_df' as the base dataframe to merge to
full_df = corr_coreg_proofread_df.copy()

# merge the lateral distance 
full_df = pd.merge(full_df, lateral_distance_proofread_df, on=["pre_pt_root_id", "post_pt_root_id"])

# merge the connectivity, setting empty entries to 0
full_df = pd.merge(full_df, conn_df, on=["pre_pt_root_id", "post_pt_root_id"], how="left").fillna(0)

In [ ]:
full_df["connected"] = np.array(full_df["size"] > 0)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<b>Measurement:</b> We will summarize the structure-function result with one value, the mean correlation of the cells in the connected and overall group.

In [ ]:
full_df.groupby("connected")[stimulus_conditions].mean()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<b>Distance-based null model to sample connectomes:</b> Instead of using the not-connected neurons as our NULL model for comparison, we want to use our insight about the connection probabilities vs neuron distance to create a stronger NULL model. We will achieve this by sampling new "connectomes" based on the distance-based connection probabilities. For each new connectome, we will calculate the correlation of the connected (sampled) neurons.

First, we assign probabilities to each potential connection by recreating the connection probability measurements from #1:

In [ ]:
conn_dist_df = pd.merge(conn_df, lateral_distance_df, on=["pre_pt_root_id", "post_pt_root_id"])

In [ ]:
# Distance vs Connectivity
distance_bins = np.linspace(0, 500, 101)

proofread_pair_counts = np.histogram(lateral_distance_proofread_df["distance"], distance_bins)[0]
connected_pair_counts = np.histogram(conn_dist_df["distance"], distance_bins)[0]

connection_probability = connected_pair_counts / proofread_pair_counts
connection_probability_df = pd.DataFrame({"bin_id": np.arange(len(distance_bins)-1), "bin_start": distance_bins[:-1], "bin_end": distance_bins[1:], "connection_probability": connection_probability})

In [ ]:
connection_probability_df.head(5)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

Finally, we assign each neuron-neuron pair their probability to be connected based on their distance. "np.digitize" allows us to assign each pair to a bin, which we then use to combine the "full_df" with the connection probability dataframe.

In [ ]:
full_df["bin_id"] = np.digitize(full_df["distance"], distance_bins)
full_df = pd.merge(full_df, connection_probability_df, on="bin_id")

In [ ]:
full_df.head(5)

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

<b>Task:</b> Sampling connectomes. Sample a connectome using  [np.random.choice](https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html). This function takes a list of entries ("a"), their sampling probabilities which have to sum to one as input ("p"), and a number of samples that should be taken ("size").

</div>

In [ ]:
full_sampled_df = full_df.copy()
sampled_idx = np.random.choice(a= , size= , replace= , p= )

full_sampled_df["connected_sampled"] = False
full_sampled_df.loc[sampled_idx, "connected_sampled"] = True

full_sampled_df.loc[sampled_idx]

In [ ]:
full_sampled_df.groupby("connected_sampled")[stimulus_conditions].mean()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

We repeat the sampling 10,000 times and combine the result in a DataFrame.

In [ ]:
sampled_means = []

for _ in tqdm.tqdm(range(10_000)):
    full_sampled_df = full_df.copy()
    sampled_idx = np.random.choice(
        full_sampled_df.index,
        p=full_sampled_df["connection_probability"] / full_sampled_df["connection_probability"].sum(),
        size=np.sum(full_sampled_df["connected"])
    )
    
    full_sampled_df["connected_sampled"] = False
    full_sampled_df.loc[sampled_idx, "connected_sampled"] = True
    
    means = full_sampled_df.loc[full_sampled_df["connected_sampled"], stimulus_conditions].mean()
    sampled_means.append(means)

# Combine into a DataFrame (each row = one sample)
sampled_means_df = pd.DataFrame(sampled_means)

In [ ]:
sampled_means_df.head(5)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

With the results in hand, lets plot our distribution for a single stimulus condition.

In [ ]:
stim = stimulus_conditions[2]
stim

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))

ax.set_title(stim, fontsize=18)

sns.histplot(data=sampled_means_df, x=stim, stat="percent", bins=101, element="step", label="Distance-based\nconnected samples", 
             lw=3, fill=True, alpha=.5, color="k")

ax.legend(frameon=False, fontsize=14, bbox_to_anchor=[1, 1], loc="upper left")

ax.tick_params(axis='both', which='major', labelsize=15, width=2, length=8)  # Set larger tick labels
ax.tick_params(axis='both', which='minor', labelsize=14, width=1, length=4)  # Set larger tick labels

ax.spines['top'].set_visible(False)  # Hide the top spine
ax.spines['right'].set_visible(False)  # Hide the right spine
ax.spines['bottom'].set_linewidth(2)  # Hide the top spine
ax.spines['left'].set_linewidth(2)  # Hide the right spine

ax.set_xlabel("Total trace correlation during stimulus", fontsize=16)
ax.set_ylabel("Fraction of neuron pairs", fontsize=15)
sns.despine(offset=15, trim=False)
plt.tight_layout()

plt.show()

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

<p><b>Task:</b> How different is our distribution from the observed correlation of the connected neurons? Calculate the Z-Score between the observed result and the measured correlation of the connected neurons. Use df.mean() and df.std().

</div>

In [ ]:
observed_stim_means = full_df[full_df["connected"]][stimulus_conditions].mean()

# calculate z scores: using df.mean() and df.std()

In [ ]:
zscores

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

Visualizing it all in one plot:

In [ ]:
stim = stimulus_conditions[2]
stim

In [ ]:
zscore = (full_df[full_df["connected"]][stim].mean() - sampled_means_df[stim].mean()) / sampled_means_df[stim].std()

fig, ax = plt.subplots(figsize=(12, 5))

ax.set_title(stim, fontsize=18)

sns.histplot(data=sampled_means_df, x=stim, stat="percent", bins=101, element="step", label="Distance-based\nconnected samples", 
             lw=3, fill=True, alpha=.5, color="k")

ax.axvline(x=full_df[full_df["connected"]][stim].mean(), color='r', linestyle='--', label='Observed:\nConnected', lw=3)
ax.axvline(x=full_df[~full_df["connected"]][stim].mean(), color='b', linestyle='--', label='Observed:\nNot connected', lw=3)

ax.text(ax.get_xlim()[0], ax.get_ylim()[1] * .95, f"Z-Score: {zscore:.2f}", fontsize=14)
ax.legend(frameon=False, fontsize=14, bbox_to_anchor=[1, 1], loc="upper left")

ax.tick_params(axis='both', which='major', labelsize=15, width=2, length=8)  # Set larger tick labels
ax.tick_params(axis='both', which='minor', labelsize=14, width=1, length=4)  # Set larger tick labels

ax.spines['top'].set_visible(False)  # Hide the top spine
ax.spines['right'].set_visible(False)  # Hide the right spine
ax.spines['bottom'].set_linewidth(2)  # Hide the top spine
ax.spines['left'].set_linewidth(2)  # Hide the right spine

ax.set_xlabel("Total trace correlation during stimulus", fontsize=16)
ax.set_ylabel("Fraction of neuron pairs", fontsize=15)
sns.despine(offset=15, trim=False)
plt.tight_layout()

plt.show()